In [ ]:
!pip install agno fastembed qdrant-client

In [ ]:
!pip install tavily-python

In [1]:
import os

from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get("GOOGLE_API_KEY")
os.environ['TAVILY_API_KEY'] = userdata.get("TAVILY_API_KEY")

In [2]:
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools.tavily import TavilyTools

In [3]:
url = userdata.get("QDRANT_URL")
api_key = userdata.get("QDRANT_API_KEY")

## Custom Tools

In [44]:
from qdrant_client import QdrantClient,models
from qdrant_client.models import PointStruct
from fastembed import TextEmbedding, SparseTextEmbedding

In [48]:
collection_name = "hybrid"
client = QdrantClient(
    url = url,
    api_key=api_key,
)

In [46]:
dense_embedding_model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-base-en")

In [47]:
sparse_embedding_model = SparseTextEmbedding("Qdrant/BM25")

In [58]:
def knowledge_base_tool(query: str) -> str:
  """
  Use this function to get relevant information from the knowledge base.

  Args:
       query: user prompt

  Returns:
    context (str): retrieve the answer from the knowledge base for the user query.
  """
  dense_vectors = next(dense_embedding_model.query_embed(query))
  sparse_vectors = next(sparse_embedding_model.query_embed(query))

  prefetch = [
    models.Prefetch(
        query = dense_vectors,
        using = "dense",
        limit = 15
    ),
    models.Prefetch(
        query = models.SparseVector(**sparse_vectors.as_object()),
        using = "sparse",
        limit = 15
    )]


  relevant_docs = client.query_points(
    collection_name = collection_name,
    prefetch = prefetch,
    query = dense_vectors,using = "dense",
    with_payload=True,limit = 3,
    )

  context = []
  for info in relevant_docs.points:
    context.append(info.payload['document'])

  return ".".join(context)

In [4]:
instructions = """
You are an expert routing Agent, you first need to check the knowledge_base_tool and check if its relevant
If user query is not relevant to the context or knowledge_base_tool then use search_tool.

Now If the user query overlaps and is not retrieved from the Knowledge_base_tool that means the information is not present in the knowledge_base.
Then you need to trigger the search tool to get the results from web via Tavily Search_tool.
"""

In [68]:
agent = Agent(
    role = "Expert QA Assistant",
    model=Gemini(id="gemini-2.5-pro"),
    tools=[TavilyTools(),knowledge_base_tool],
    instructions=instructions,
    markdown=True,
)

In [69]:
query = "mail to contact Atyantik"

In [70]:
agent.print_response(query,stream=True)

Output()

In [71]:
query2 = "who is Virat Kohli?"

In [72]:
agent.print_response(query2,stream=True)

Output()

## Knowledge

In [ ]:
!pip install pypdf

In [22]:
from agno.knowledge.knowledge import Knowledge
from agno.vectordb.qdrant import Qdrant
from agno.vectordb.search import SearchType
from agno.knowledge.embedder.fastembed import FastEmbedEmbedder

In [23]:
collection_name = "agentic_rag"

In [24]:
db = Qdrant(
    collection=collection_name,
    url=url,
    api_key=api_key,
    embedder = FastEmbedEmbedder(),
    search_type=SearchType.hybrid,
)

In [34]:
knowledge_base = Knowledge(
    vector_db=db,
    #max_results=5
)

In [26]:
import nest_asyncio
nest_asyncio.apply()

In [27]:
knowledge_base.add_content(
        path="/content/Emergent-Build-Guide.pdf",
    )

INFO skip_if_exists is disabled, disabling upsert

INFO Loading content: f1856fa8-9e20-52c0-af5b-320ccb109b19

INFO Adding content from path, f1856fa8-9e20-52c0-af5b-320ccb109b19, None, /content/Emergent-Build-Guide.pdf, None

INFO Adding file /content/Emergent-Build-Guide.pdf due to include/exclude filters

INFO Error checking if content_hash 30fdf2e74f923ce0ae8a8407e42024aa77b2c04660a488185b253f8f3a1a8707 exists:       
     Unexpected Response: 400 (Bad Request)                                                                        
     Raw response content:                                                                                         
     b'{"status":{"error":"Bad request: Index required but not found for \\"content_hash\\" of one of the following
     types: [keyword]. Help: Create an index for this key or use a different filter."},"time":5 ...'

...

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

WARNING  Contents DB not found for knowledge base

In [31]:
agent = Agent(
    role = "Expert QA Assistant",
    model=Gemini(id="gemini-2.5-flash"),
    tools=[TavilyTools()],
    knowledge=knowledge_base,
    search_knowledge=True,
    instructions=instructions,
    markdown=True,
)

In [32]:
query = "tips on how to implement features"

In [33]:
agent.print_response(query,stream=True)

Output()

INFO Found 10 documents

In [ ]:
"""
from agno.knowledge.chunking.agentic import AgenticChunking
from agno.knowledge.reader.pdf_reader import PDFReader


knowledge_base.add_content(
    path="/content/Emergent-Build-Guide.pdf",
    reader=PDFReader(
        name="Agentic Chunking Reader",
        chunking_strategy=AgenticChunking(),
    ),
))

import asyncio
asyncio.run(knowledge_base.add_content_async(
    path="/content/Emergent-Build-Guide.pdf",
    reader=PDFReader(
        name="Agentic Chunking Reader",
        chunking_strategy=AgenticChunking(),
    ),
))
"""